In [1]:
import random

class Trader:
    """Classe de base pour tous les traders."""
    def __init__(self, trader_id, balance, leverage):
        self.trader_id = trader_id
        self.balance = balance
        self.leverage = leverage
        self.position = None  # 'BUY', 'SELL', ou None
        self.entry_price = None
        self.liquidation_price = None
        self.margin = balance / leverage
        self.is_liquidated = False

    def open_position(self, position, price):
        """Ouvre une position en fonction du prix du marché."""
        if self.is_liquidated:
            return
        self.position = position
        self.entry_price = price
        self.liquidation_price = self.calculate_liquidation_price()

    def calculate_liquidation_price(self):
        """Calcule le prix auquel ce trader sera liquidé."""
        if self.position == 'BUY':
            return self.entry_price * (1 - (self.margin / (self.balance * self.leverage)))
        elif self.position == 'SELL':
            return self.entry_price * (1 + (self.margin / (self.balance * self.leverage)))
        return None

    def check_liquidation(self, market_price):
        """Vérifie si le trader doit être liquidé."""
        if self.position and not self.is_liquidated:
            if (self.position == 'BUY' and market_price <= self.liquidation_price) or \
               (self.position == 'SELL' and market_price >= self.liquidation_price):
                self.is_liquidated = True
                self.position = None
                self.entry_price = None
                self.liquidation_price = None
                print(f"Trader {self.trader_id} liquidated at price {market_price}")

class MarketMaker(Trader):
    """Trader Market Maker - Fournit de la liquidité."""
    def __init__(self, trader_id, balance):
        super().__init__(trader_id, balance, leverage=random.choice([2, 5]))
        self.herding_prob = 0.1  # Faible mimétisme

    def place_limit_orders(self):
        """Place des ordres limites pour fournir de la liquidité."""
        print(f"Market Maker {self.trader_id} place un ordre limite.")

class DegenTrader(Trader):
    """Trader Degen - Très spéculatif et risqué."""
    def __init__(self, trader_id, balance):
        super().__init__(trader_id, balance, leverage=random.choice([10, 20, 50]))
        self.herding_prob = 0.8  # Très influencé par les autres

    def panic_sell(self, market_price):
        """Vend rapidement en cas de forte baisse."""
        if self.position == 'BUY' and random.random() < self.herding_prob:
            print(f"Degen Trader {self.trader_id} panique et vend au marché !")
            self.open_position('SELL', market_price)

class SwingTrader(Trader):
    """Trader Swing - Suit les tendances macro."""
    def __init__(self, trader_id, balance):
        super().__init__(trader_id, balance, leverage=random.choice([5, 10, 20]))
        self.herding_prob = 0.5  # Moyenne sensibilité au marché

    def follow_trend(self, trend_signal, market_price):
        """Suit la tendance du marché (BUY si haussier, SELL si baissier)."""
        if trend_signal == 'bullish':
            self.open_position('BUY', market_price)
        elif trend_signal == 'bearish':
            self.open_position('SELL', market_price)

class ArbitrageTrader(Trader):
    """Trader Arbitrage - Exploite les écarts de prix."""
    def __init__(self, trader_id, balance):
        super().__init__(trader_id, balance, leverage=random.choice([2, 5, 10]))
        self.herding_prob = 0.2  # Faible mimétisme

    def detect_arbitrage_opportunity(self, market_price, external_price):
        """Identifie les opportunités d'arbitrage entre deux marchés."""
        if abs(market_price - external_price) > 1:
            print(f"Arbitrage Trader {self.trader_id} exploite une différence de prix !")

# Génération de 20 traders avec différents profils
trader_profiles = [MarketMaker, DegenTrader, SwingTrader, ArbitrageTrader]
traders = [random.choice(trader_profiles)(i, random.randint(1000, 50000)) for i in range(20)]

# Affichage des traders créés
for trader in traders:
    print(f"Trader {trader.trader_id}: {trader.__class__.__name__}, Balance: {trader.balance}, Leverage: {trader.leverage}, Herding Prob: {trader.herding_prob}")


Trader 0: SwingTrader, Balance: 27828, Leverage: 20, Herding Prob: 0.5
Trader 1: DegenTrader, Balance: 5815, Leverage: 10, Herding Prob: 0.8
Trader 2: MarketMaker, Balance: 30917, Leverage: 5, Herding Prob: 0.1
Trader 3: SwingTrader, Balance: 45858, Leverage: 20, Herding Prob: 0.5
Trader 4: DegenTrader, Balance: 21887, Leverage: 10, Herding Prob: 0.8
Trader 5: ArbitrageTrader, Balance: 13054, Leverage: 10, Herding Prob: 0.2
Trader 6: SwingTrader, Balance: 6355, Leverage: 20, Herding Prob: 0.5
Trader 7: MarketMaker, Balance: 21147, Leverage: 2, Herding Prob: 0.1
Trader 8: SwingTrader, Balance: 17811, Leverage: 10, Herding Prob: 0.5
Trader 9: ArbitrageTrader, Balance: 29549, Leverage: 10, Herding Prob: 0.2
Trader 10: SwingTrader, Balance: 29588, Leverage: 5, Herding Prob: 0.5
Trader 11: SwingTrader, Balance: 41226, Leverage: 5, Herding Prob: 0.5
Trader 12: MarketMaker, Balance: 30560, Leverage: 5, Herding Prob: 0.1
Trader 13: SwingTrader, Balance: 5977, Leverage: 20, Herding Prob: 0.5
Tr

In [2]:
import pandas as pd
import random
import hashlib
from datetime import datetime

class Order:
    """Représente un ordre dans le carnet d'ordres."""
    def __init__(self, trader, order_type, price, quantity):
        self.order_hash = self.generate_order_hash()
        self.block = self.get_current_block()
        self.action = "EVENT_NEW"
        self.price = price
        self.quantity = quantity
        self.order_type = order_type  # 'BUY_PO', 'SELL_PO', 'BUY', 'SELL'
        self.subaccount = trader.trader_id

    def generate_order_hash(self):
        """Génère un hash unique pour l'ordre."""
        return hashlib.sha256(str(random.randint(0, 1e9)).encode()).hexdigest()[:10]
    
    def get_current_block(self):
        """Simule un bloc de transaction basé sur le timestamp."""
        return int(datetime.timestamp(datetime.now()))

class OrderBook:
    """Carnet d'ordres gérant les transactions."""
    def __init__(self):
        self.orders = []
    
    def add_order(self, order):
        """Ajoute un nouvel ordre au carnet."""
        self.orders.append(order)
    
    def get_order_book_df(self):
        """Retourne le carnet d'ordres sous forme de DataFrame pandas."""
        data = [{
            "OrderHash": order.order_hash,
            "Block": order.block,
            "Action": order.action,
            "Price": order.price,
            "Quantity": order.quantity,
            "OrderType": order.order_type,
            "Subaccount": order.subaccount
        } for order in self.orders]
        return pd.DataFrame(data)

# Génération d'ordres aléatoires à partir des traders existants
order_book = OrderBook()
order_types = ['BUY_PO', 'SELL_PO', 'BUY', 'SELL']

for trader in traders:
    price = random.randint(95000, 96000) * 1000  # Simule une fourchette de prix réaliste
    quantity = round(random.uniform(0.01, 2.0), 3)
    order_type = random.choice(order_types)
    order = Order(trader, order_type, price, quantity)
    order_book.add_order(order)

# Affichage du carnet d'ordres sous forme de DataFrame
df_order_book = order_book.get_order_book_df()

import ace_tools as tools
tools.display_dataframe_to_user(name="Order Book", dataframe=df_order_book)


TypeError: 'float' object cannot be interpreted as an integer

In [4]:
import pandas as pd
import random
import hashlib
from datetime import datetime

class Order:
    """Représente un ordre dans le carnet d'ordres."""
    def __init__(self, trader, order_type, price, quantity):
        self.order_hash = self.generate_order_hash()
        self.block = self.get_current_block()
        self.action = "EVENT_NEW"
        self.price = price
        self.quantity = quantity
        self.order_type = order_type  # 'BUY_PO', 'SELL_PO', 'BUY', 'SELL'
        self.subaccount = trader.trader_id

    def generate_order_hash(self):
        """Génère un hash unique pour l'ordre."""
        return hashlib.sha256(str(random.randint(0, int(1e9))).encode()).hexdigest()[:10]
    
    def get_current_block(self):
        """Simule un bloc de transaction basé sur le timestamp."""
        return int(datetime.timestamp(datetime.now()))

class OrderBook:
    """Carnet d'ordres gérant les transactions."""
    def __init__(self):
        self.orders = []
    
    def add_order(self, order):
        """Ajoute un nouvel ordre au carnet."""
        self.orders.append(order)
    
    def get_order_book_df(self):
        """Retourne le carnet d'ordres sous forme de DataFrame pandas."""
        data = [{
            "OrderHash": order.order_hash,
            "Block": order.block,
            "Action": order.action,
            "Price": order.price,
            "Quantity": order.quantity,
            "OrderType": order.order_type,
            "Subaccount": order.subaccount
        } for order in self.orders]
        return pd.DataFrame(data)

# Génération d'ordres aléatoires à partir des traders existants
order_book = OrderBook()
order_types = ['BUY_PO', 'SELL_PO', 'BUY', 'SELL']

for trader in traders:
    price = random.randint(95000, 96000) * 1000  # Simule une fourchette de prix réaliste
    quantity = round(random.uniform(0.01, 2.0), 3)
    order_type = random.choice(order_types)
    order = Order(trader, order_type, price, quantity)
    order_book.add_order(order)

# Affichage du carnet d'ordres sous forme de DataFrame
df_order_book = order_book.get_order_book_df()

import ace_tools as tools
tools.display_dataframe_to_user(name="Order Book", dataframe=df_order_book)

ModuleNotFoundError: No module named 'ace_tools'

In [12]:
import pandas as pd
import random
import hashlib
from datetime import datetime

#class Order:
    """Représente un ordre dans le carnet d'ordres."""
    def __init__(self, trader, order_type, price, quantity):
        self.order_hash = self.generate_order_hash()
        self.block = self.get_current_block()
        self.action = "EVENT_NEW"
        self.price = price
        self.quantity = quantity
        self.order_type = order_type  # 'BUY_PO', 'SELL_PO', 'BUY', 'SELL'
        self.subaccount = trader.trader_id

    def generate_order_hash(self):
        """Génère un hash unique pour l'ordre."""
        return hashlib.sha256(str(random.randint(0, int(1e9))).encode()).hexdigest()[:10]
    
    def get_current_block(self):
        """Simule un bloc de transaction basé sur le timestamp."""
        return int(datetime.timestamp(datetime.now()))

#class OrderBook:
    """Carnet d'ordres gérant les transactions."""
    def __init__(self):
        self.orders = []
    
    def add_order(self, order):
        """Ajoute un nouvel ordre au carnet."""
        self.orders.append(order)
    
    def get_order_book_df(self):
        """Retourne le carnet d'ordres sous forme de DataFrame pandas."""
        data = [{
            "OrderHash": order.order_hash,
            "Block": order.block,
            "Action": order.action,
            "Price": order.price,
            "Quantity": order.quantity,
            "OrderType": order.order_type,
            "Subaccount": order.subaccount
        } for order in self.orders]
        return pd.DataFrame(data)

# Génération d'ordres aléatoires à partir des traders existants
order_book = OrderBook()
order_types = ['BUY_PO', 'SELL_PO', 'BUY', 'SELL']

for trader in traders:
    price = random.randint(95000, 96000) * 1000  # Simule une fourchette de prix réaliste
    quantity = round(random.uniform(0.01, 2.0), 3)
    order_type = random.choice(order_types)
    order = Order(trader, order_type, price, quantity)
    order_book.add_order(order)

# Affichage du carnet d'ordres sous forme de DataFrame
df_order_book = order_book.get_order_book_df()

print(df_order_book)


     OrderHash       Block     Action     Price  Quantity OrderType  \
0   fc4d083d5a  1738157715  EVENT_NEW  95124000     1.558       BUY   
1   a9ad050d0c  1738157715  EVENT_NEW  95795000     0.945   SELL_PO   
2   f9b56745ba  1738157715  EVENT_NEW  95052000     1.414   SELL_PO   
3   b0c1bcb220  1738157715  EVENT_NEW  95119000     0.169    BUY_PO   
4   15579be969  1738157715  EVENT_NEW  95318000     1.483      SELL   
5   4effe89fa5  1738157715  EVENT_NEW  95370000     0.319      SELL   
6   e167cb5531  1738157715  EVENT_NEW  95342000     1.325   SELL_PO   
7   5855200864  1738157715  EVENT_NEW  95147000     0.076      SELL   
8   6a0ccd226f  1738157715  EVENT_NEW  95605000     0.407      SELL   
9   60765b4c07  1738157715  EVENT_NEW  95473000     1.188       BUY   
10  7bb8f9b051  1738157715  EVENT_NEW  95325000     1.809    BUY_PO   
11  fbdc93062e  1738157715  EVENT_NEW  95608000     1.293       BUY   
12  de45f6fc52  1738157715  EVENT_NEW  95090000     1.310       BUY   
13  29

In [39]:
import numpy as np

class Market:
    """Classe représentant le marché et la mise à jour du prix."""
    def __init__(self, initial_price=96000000, alpha=0.0001, beta=0.0002):
        self.price = initial_price  # Prix initial du marché
        self.alpha = alpha  # Coefficient d'impact des ordres sur le prix
        self.beta = beta  # Coefficient d'impact des liquidations sur le prix
    
    def update_price(self, order_book):
        """Met à jour le prix du marché en utilisant la loi de la racine carrée."""
        df_orders = order_book.get_order_book_df()
        
        buy_volume = df_orders[df_orders['OrderType'].str.startswith('BUY')]['Quantity'].sum()
        sell_volume = df_orders[df_orders['OrderType'].str.startswith('SELL')]['Quantity'].sum()
        liquidation_volume = df_orders[df_orders['Action'] == 'EVENT_LIQ']['Quantity'].sum() if 'EVENT_LIQ' in df_orders['Action'].values else 0
        
        net_order_flow = buy_volume - sell_volume
        
        # Application de la loi de la racine carrée
        price_change = (self.alpha * np.sqrt(abs(net_order_flow))) - (self.beta * np.sqrt(abs(liquidation_volume)))
        self.price *= (1 + price_change)
        
        print(f"New Market Price: {self.price:.2f}")
        return self.price

# Initialisation du marché
market = Market()
new_price = market.update_price(order_book)

New Market Price: 96027379.36


In [38]:
import pandas as pd
import random
import hashlib
from datetime import datetime

class Order:
    """Représente un ordre dans le carnet d'ordres."""
    def __init__(self, trader, order_type, price, quantity, market_price):
        self.order_hash = self.generate_order_hash()
        self.block = self.get_current_block()
        self.action = "EVENT_NEW"
        self.price = self.adjust_price(price, market_price)
        self.quantity = quantity
        self.order_type = order_type  # 'BUY_PO', 'SELL_PO', 'BUY', 'SELL'
        self.subaccount = trader.trader_id

    def generate_order_hash(self):
        """Génère un hash unique pour l'ordre."""
        return hashlib.sha256(str(random.randint(0, int(1e9))).encode()).hexdigest()[:10]
    
    def get_current_block(self):
        """Simule un bloc de transaction basé sur le timestamp."""
        return int(datetime.timestamp(datetime.now()))
    
    def adjust_price(self, base_price, market_price):
        """Ajuste le prix de l'ordre en fonction du prix du marché."""
        deviation = random.uniform(-0.002, 0.002)  # Variation de +/- 0.2%
        return market_price * (1 + deviation)

class OrderBook:
    """Carnet d'ordres gérant les transactions et influencé par le prix du marché."""
    def __init__(self, market):
        self.orders = []
        self.market = market  # Référence au marché
    
    def add_order(self, trader, order_type, quantity):
        """Ajoute un nouvel ordre influencé par le prix du marché actuel."""
        order = Order(trader, order_type, self.market.price, quantity, self.market.price)
        self.orders.append(order)
    
    def get_order_book_df(self):
        """Retourne le carnet d'ordres sous forme de DataFrame pandas."""
        data = [{
            "OrderHash": order.order_hash,
            "Block": order.block,
            "Action": order.action,
            "Price": order.price,
            "Quantity": order.quantity,
            "OrderType": order.order_type,
            "Subaccount": order.subaccount
        } for order in self.orders]
        return pd.DataFrame(data)

# Initialisation du carnet d'ordres
order_book = OrderBook(market)
order_types = ['BUY_PO', 'SELL_PO', 'BUY', 'SELL']

# Génération d'ordres influencés par le prix du marché
for trader in traders:
    quantity = round(random.uniform(0.01, 2.0), 3)
    order_type = random.choice(order_types)
    order_book.add_order(trader, order_type, quantity)

# Affichage du carnet d'ordres sous forme de DataFrame
df_order_book = order_book.get_order_book_df()
print(df_order_book)


     OrderHash       Block     Action         Price  Quantity OrderType  \
0   a52206cc35  1738159880  EVENT_NEW  9.582703e+07     0.867      SELL   
1   ba26aa3100  1738159880  EVENT_NEW  9.613117e+07     1.186    BUY_PO   
2   7344491a40  1738159880  EVENT_NEW  9.603438e+07     1.598       BUY   
3   a7576a937b  1738159880  EVENT_NEW  9.620700e+07     0.993       BUY   
4   7115e10089  1738159880  EVENT_NEW  9.619529e+07     0.196      SELL   
5   44477d1450  1738159880  EVENT_NEW  9.603394e+07     0.138    BUY_PO   
6   e0402f425e  1738159880  EVENT_NEW  9.599651e+07     1.826      SELL   
7   100e052938  1738159880  EVENT_NEW  9.586769e+07     0.231      SELL   
8   fc67239f99  1738159880  EVENT_NEW  9.596054e+07     0.607    BUY_PO   
9   7c9021dfd8  1738159880  EVENT_NEW  9.604091e+07     1.364       BUY   
10  e438e92b92  1738159880  EVENT_NEW  9.596273e+07     0.095       BUY   
11  bad3b5e997  1738159880  EVENT_NEW  9.586831e+07     0.059   SELL_PO   
12  05544e87c5  173815988

In [42]:
import pandas as pd
import random
import hashlib
from datetime import datetime

class Order:
    """Représente un ordre dans le carnet d'ordres."""
    def __init__(self, trader, order_type, price, quantity, market_price):
        self.order_hash = self.generate_order_hash()
        self.block = self.get_current_block()
        self.action = "EVENT_NEW"
        self.price = self.adjust_price(price, market_price)
        self.quantity = quantity
        self.order_type = order_type  # 'BUY_PO', 'SELL_PO', 'BUY', 'SELL'
        self.subaccount = trader.trader_id

    def generate_order_hash(self):
        """Génère un hash unique pour l'ordre."""
        return hashlib.sha256(str(random.randint(0, int(1e9))).encode()).hexdigest()[:10]
    
    def get_current_block(self):
        """Simule un bloc de transaction basé sur le timestamp."""
        return int(datetime.timestamp(datetime.now()))
    
    def adjust_price(self, base_price, market_price):
        """Ajuste le prix de l'ordre en fonction du prix du marché."""
        deviation = random.uniform(-0.002, 0.002)  # Variation de +/- 0.2%
        return market_price * (1 + deviation)

class OrderBook:
    """Carnet d'ordres gérant les transactions et influencé par le prix du marché."""
    def __init__(self, market):
        self.orders = []
        self.market = market  # Référence au marché
    
    def add_order(self, trader, order_type, quantity):
        """Ajoute un nouvel ordre influencé par le prix du marché actuel."""
        order = Order(trader, order_type, self.market.price, quantity, self.market.price)
        self.orders.append(order)
    
    def process_orders(self):
        """Exécute les ordres en attente et nettoie le carnet d'ordres."""
        executed_orders = []
        for order in self.orders:
            if order.order_type in ['BUY', 'SELL']:  # Ordres au marché exécutés immédiatement
                executed_orders.append(order)
            elif order.order_type == 'BUY_PO' and order.price >= self.market.price:
                executed_orders.append(order)
            elif order.order_type == 'SELL_PO' and order.price <= self.market.price:
                executed_orders.append(order)
        
        # Suppression des ordres exécutés du carnet
        for executed_order in executed_orders:
            self.orders.remove(executed_order)
            print(f"Order {executed_order.order_hash} executed at market price {self.market.price}")
    
    def get_order_book_df(self):
        """Retourne le carnet d'ordres sous forme de DataFrame pandas."""
        data = [{
            "OrderHash": order.order_hash,
            "Block": order.block,
            "Action": order.action,
            "Price": order.price,
            "Quantity": order.quantity,
            "OrderType": order.order_type,
            "Subaccount": order.subaccount
        } for order in self.orders]
        return pd.DataFrame(data)

# Initialisation du carnet d'ordres
order_book = OrderBook(market)
order_types = ['BUY_PO', 'SELL_PO', 'BUY', 'SELL']

# Génération d'ordres influencés par le prix du marché
for trader in traders:
    quantity = round(random.uniform(0.01, 2.0), 3)
    order_type = random.choice(order_types)
    order_book.add_order(trader, order_type, quantity)

# Affichage du carnet d'ordres sous forme de DataFrame
df_order_book = order_book.get_order_book_df()
print(df_order_book)



     OrderHash       Block     Action         Price  Quantity OrderType  \
0   cf7da09fd6  1738160410  EVENT_NEW  9.604378e+07     0.216    BUY_PO   
1   60fcdb6d47  1738160410  EVENT_NEW  9.610357e+07     0.442       BUY   
2   e368a43abc  1738160410  EVENT_NEW  9.624609e+07     1.278       BUY   
3   1fac649cf5  1738160410  EVENT_NEW  9.587648e+07     0.930       BUY   
4   efb8c31252  1738160410  EVENT_NEW  9.599990e+07     1.280       BUY   
5   2d724891f5  1738160410  EVENT_NEW  9.608645e+07     1.830      SELL   
6   8b22056884  1738160410  EVENT_NEW  9.595318e+07     1.201      SELL   
7   5b1b2b25a0  1738160410  EVENT_NEW  9.596760e+07     1.463      SELL   
8   674c9bb25e  1738160410  EVENT_NEW  9.616187e+07     1.472      SELL   
9   9dd559015e  1738160410  EVENT_NEW  9.598758e+07     1.820   SELL_PO   
10  fc8fb48174  1738160410  EVENT_NEW  9.621074e+07     0.139    BUY_PO   
11  636b6a95d4  1738160410  EVENT_NEW  9.599684e+07     0.479      SELL   
12  23360a6a81  173816041

In [43]:
import time

class TradingSimulation:
    """Classe principale qui orchestre la mise à jour du marché et la gestion du carnet d'ordres."""
    def __init__(self, market, order_book, traders, max_ticks=100, tick_duration=1):
        self.market = market
        self.order_book = order_book
        self.traders = traders
        self.max_ticks = max_ticks  # Nombre de cycles de simulation
        self.tick_duration = tick_duration  # Temps entre chaque cycle
        self.current_tick = 0
    
    def run_simulation(self):
        """Démarre la simulation de trading."""
        while self.current_tick < self.max_ticks:
            print(f"\n--- Tick {self.current_tick + 1} ---")
            
            # 1. Génération de nouveaux ordres
            for trader in self.traders:
                quantity = round(random.uniform(0.01, 2.0), 3)
                order_type = random.choice(['BUY_PO', 'SELL_PO', 'BUY', 'SELL'])
                self.order_book.add_order(trader, order_type, quantity)
            
            # 2. Mise à jour du prix du marché
            self.market.update_price(self.order_book)
            
            # 3. Exécution des ordres et nettoyage du carnet
            self.order_book.process_orders(self.market)
            
            # 4. Affichage du marché et du carnet d'ordres
            print(f"Updated Market Price: {self.market.price:.2f}")
            print(self.order_book.get_order_book_df().head())
            
            self.current_tick += 1
            time.sleep(self.tick_duration)  # Pause pour simuler un marché en temps réel

# Initialisation de la simulation
simulation = TradingSimulation(market, order_book, traders)
simulation.run_simulation()


--- Tick 1 ---
New Market Price: 96074708.03


TypeError: OrderBook.process_orders() takes 1 positional argument but 2 were given